In [64]:
import numpy as np; import pandas as pd; import os 
from d6_functions import *
from RL_functions import *
from IPython.display import clear_output

path = 'RL_Data/01_RBC/03_Data/'

data_files = os.listdir(path)
price = {'energy': 0.0984, 't_disc': 0.8, 'dr_price': 7.89 }

'''Define empty dataframe '''
df_all = pd.DataFrame()

for file in data_files:      
     data_pd = pd.read_csv(path+file,converters={'action': pd.eval})  
     data_pd['next_state'] = data_pd['state'].shift(-1)
     data_pd['act'] = data_pd['action'].map(lambda x: x[0])
     data_pd = data_pd[:-1]   
     df_all = pd.concat([df_all, data_pd])


In [65]:
def get_w(oa_air,hours,dr_start_time,dr_end_time):
     if oa_air<21:
          dif = (21-oa_air)
          s =  dif
     else:
          s=1

     if (hours >= dr_start_time) & (hours< dr_end_time):                       
          dr_power_limit = 9000 
          w = [-100, -1, -50]               
     else:                        
          dr_power_limit = 30000
          w = [-200*s, -1, 0]

     return w  
     

In [ ]:
len(df_all)

In [66]:
rewards = []

for i in range(len(df_all)):
     print ("percentage: {}".format(i/len(df_all)))
     hours = df_all.iloc[i].senHouDec_y
     hvac_energy = df_all.iloc[i].hvac_energy_Z0+df_all.iloc[i].hvac_energy_Z1+df_all.iloc[i].hvac_energy_Z2+df_all.iloc[i].hvac_energy_Z3+df_all.iloc[i].hvac_energy_Z4
     tdisc = df_all.iloc[i].thermal_discomfort_Z0+df_all.iloc[i].thermal_discomfort_Z1+df_all.iloc[i].thermal_discomfort_Z2+df_all.iloc[i].thermal_discomfort_Z3+df_all.iloc[i].thermal_discomfort_Z4
     ppen = df_all.iloc[i].power_pen
     hours = df_all.iloc[i].senHouDec_y
     dr_start_time = df_all.iloc[i].dr_start_time
     dr_end_time = df_all.iloc[i].dr_end_time
     oa_air = df_all.iloc[i].senTemOA_y - 273.15

     occupancy  = get_occupancy(hours)

     energy_cost =  price['energy'] * hvac_energy
     tdisc_cost = price['t_disc'] * tdisc * occupancy 
     ppen_cost = price['dr_price'] * ppen

     w = get_w(oa_air,hours,dr_start_time,dr_end_time)
     reward_dqn = w[0]*energy_cost + w[1]*tdisc_cost+ w[2]*ppen_cost
     rewards.append(reward_dqn)
     # clear_output(wait=False)

clear_output(wait=False)


In [38]:
df_all['rewards_rbc'] = rewards

In [51]:
df_all.to_csv(path+'offline_rbc_df.csv')

0      [0, 0, 0, 0, 0, 0.15, 0.15, 0.15, 0.15, 0.15, ...
1      [0, 0, 0, 0, 0, 0.15, 0.15, 0.15, 0.15, 0.15, ...
2      [0, 0, 0, 0, 0, 0.15, 0.15, 0.15, 0.15, 0.15, ...
3      [0, 0, 0, 0, 0, 0.15, 0.15, 0.15, 0.15, 0.15, ...
4      [0, 0, 0, 0, 0, 0.15, 0.15, 0.15, 0.15, 0.15, ...
                             ...                        
282    [0, 0, 0, 0, 0, 0.15, 0.15, 0.15, 0.15, 0.15, ...
283    [0, 0, 0, 0, 0, 0.15, 0.15, 0.15, 0.15, 0.15, ...
284    [0, 0, 0, 0, 0, 0.15, 0.15, 0.15, 0.15, 0.15, ...
285    [0, 0, 0, 0, 0, 0.15, 0.15, 0.15, 0.15, 0.15, ...
286    [0, 0, 0, 0, 0, 0.15, 0.15, 0.15, 0.15, 0.15, ...
Name: action, Length: 287, dtype: object

In [63]:
data_pd['act'] = data_pd['action'].map(lambda x: x[0])
data_pd['act']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
26685    0.0
26686    0.0
26687    0.0
26688    0.0
26689    0.0
Name: act, Length: 26690, dtype: float64